# Demo 7

In [ ]:
from pennylane import qaoa
from pennylane import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import pennylane as qml

## Define graph

In [ ]:
edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
graph = nx.Graph(edges)
positions = nx.spring_layout(graph)

nx.draw(graph, with_labels=True, pos=positions)
plt.show()

### Two options: nodes 0 and 2, or nodes 1 and 2

## Load cost and mixer Hamiltonians

In [ ]:
cost_h, mixer_h = qaoa.min_vertex_cover(graph, constrained=False)

print("Cost Hamiltonian", cost_h)
print("Mixer Hamiltonian", mixer_h)

## Define QAOA layer

In [ ]:
def qaoa_layer(gamma, alpha):
    qaoa.cost_layer(gamma, cost_h)
    qaoa.mixer_layer(alpha, mixer_h)

In [ ]:
wires = range(4)
depth = 2

def circuit(params, **kwargs):
    for w in wires:
        qml.Hadamard(wires=w)
    qml.layer(qaoa_layer, depth, params[0], params[1])

## Load device and QNode

In [ ]:
dev = qml.device("default.qubit", wires=wires)
cost_function = qml.VQECost(circuit, cost_h, dev)

## Run QAOA

In [ ]:
optimizer = qml.GradientDescentOptimizer()
steps = 5
params = np.random.normal(0, 1, (2, 2))

for i in range(steps):
    params = optimizer.step(cost_function, params)
    print(cost_function(params))

In [ ]:
params = np.load("params.npy")

In [ ]:
print("Optimal Parameters: {}".format(params))

## Evaluate optimized state

In [ ]:
@qml.qnode(dev)
def probability_circuit(gamma, alpha):
    circuit([gamma, alpha])
    return qml.probs(wires=wires)


probs = probability_circuit(params[0], params[1])

In [ ]:
plt.style.use("seaborn")
labels = ["{0:{fill}4b}".format(i, fill='0') for i in range(len(probs))]
plt.bar(range(2 ** len(wires)), probs)
plt.xticks([i for i in range(len(probs))], labels)
plt.show()

In [ ]:
ordered_probs = sorted([(p, l) for p, l in zip(probs, labels)], reverse=True)
ordered_probs[:4]

In [ ]:
import networkx as nx

sub = graph.subgraph([0, 2])  

nx.draw(graph, pos=positions, with_labels=True)
nx.draw(sub, pos=positions, node_color="r")

In [ ]:
import networkx as nx

sub = graph.subgraph([1, 2])  

nx.draw(graph, pos=positions, with_labels=True)
nx.draw(sub, pos=positions, node_color="r")